### Loading Libraries we will use in the project

In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))

In [ ]:
import sys
!pip install dvc

In [ ]:
# import libraries
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn import preprocessing
from itertools import islice
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import scipy.stats as stat
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from six import StringIO
from IPython.display import Image  
from sklearn.tree import export_graphviz
from IPython.display import Image  

from sklearn.tree import export_graphviz
import pydotplus
from sklearn import tree


import warnings
warnings.filterwarnings('ignore')
import dvc.api

### Importing scripts & modules 

In [ ]:
from decision import DecisionTreesModel
from logestic import LogesticRegressionModel

from helper import MyHelper
import utility
import plots
Helper = MyHelper()


### Loading the data

In [ ]:
CSV_PATH = "../Data\\trainfinal.csv"
data_frame = Helper.read_csv(CSV_PATH)

In [ ]:
def get_data(tag, 
            path='data\\trainfinal.csv', 
            repo =r'C:\Users\ende\Desktop\test'
):
    rev = tag
    data_url = dvc.api.get_url(path=path, repo=repo, rev=rev)
    df = pd.read_csv(data_url)
    return df

### change categorical variables to numerical value


In [ ]:
data_frame.info()

### correcting the data types for each column

In [ ]:
data_frame['Date']=pd.to_datetime(data_frame['Date'])
data_frame['date']=pd.to_datetime(data_frame['date'])




In [ ]:
data_frame.Store.unique()

In [ ]:
data_frame.info()

In [ ]:
# change categorical variables to numerical value
lb = LabelEncoder()
data_frame['Assortment'] = lb.fit_transform(data_frame['Assortment'])
data_frame['StateHoliday'] = lb.fit_transform(data_frame['StateHoliday'])
data_frame['SchoolHoliday'] = lb.fit_transform(data_frame['SchoolHoliday'])
data_frame['StoreType'] = lb.fit_transform(data_frame['StoreType'])
data_frame['Promo2'] = lb.fit_transform(data_frame['Promo2'])
data_frame['Promo'] = lb.fit_transform(data_frame['Promo'])
data_frame['Open'] = lb.fit_transform(data_frame['Open'])





### Loading the different versions of data into separate dataframe

In [ ]:
train = get_data('v7')
#platform_df = get_data('version4')

#### 1 Model training using Decision Tree


Train Test Split


In [ ]:
# feature_cols = ["experiment", "hour", "date", 'device_make', "platform_os",  "browser"]
feature_cols = ['Store','DayOfWeek','DateCustomers','Open','Promo','StateHoliday','SchoolHoliday','StoreType','Assortment','CompetitionDistance','Promo','date']

X = train[feature_cols]
y = train[['Sale']]

test_size = 0.1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

# Model Training Using DecisionTreesModel

In [ ]:
decisionTreesModel = DecisionTreesModel(X_train, X_test,  y_train, y_test)

clf, acc_arr, loss_arr = decisionTreesModel.train(5)

Evaluating our model

In [ ]:
test_acc, loss, report, confusion_matrix = decisionTreesModel.test()
print(f"Loss on test data is: {loss:.3f}")
print()

Calculating feature importance

In [ ]:
decisionTreesModel.get_feature_importance()

## Visualzing feature importance

In [ ]:

dot_data = StringIO()

leaves_parallel=False
feature_cols = ['Store','DayOfWeek','DateCustomers','Open','Promo','StateHoliday','SchoolHoliday','StoreType','Assortment','CompetitionDistance','Promo','date']
out_put_file = "AbTestDecisionTree.dot"

dot_data = export_graphviz(clf, out_file=out_put_file,  
                filled=True, rounded=True,
                special_characters=True, feature_names = feature_cols,class_names=['Aware','Not Aware'])

graph = pydotplus.graphviz.graph_from_dot_file(out_put_file)
graph.write_png('AbTestDecisionTree.png')

Image(graph.create_png())

In [ ]:
dot_data = StringIO()

leaves_parallel=False
feature_cols = ['Store','DayOfWeek','DateCustomers','Open','Promo','StateHoliday','SchoolHoliday','StoreType','Assortment','CompetitionDistance','Promo','date']
out_put_file = "forcast.dot"

dot_data = export_graphviz(clf, out_file=out_put_file,  
                filled=True, rounded=True,
                special_characters=True, feature_names = feature_cols,class_names=['Aware','Not Aware'])

graph = pydotplus.graphviz.graph_from_dot_file(out_put_file)
graph.write_png('forcast.png')

Image(graph.create_png())

Model Building using Logestic Regression

2.1 Model training using Logestic Regression using browser dataframe

In [ ]:
feature_cols = ['Store','DayOfWeek','DateCustomers','Open','Promo','StateHoliday','SchoolHoliday','StoreType','Assortment','CompetitionDistance','Promo','date']

X = train[feature_cols]
y = train[['Sales']]

test_size = 0.1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

In [ ]:
logesticRegressionModel = LogesticRegressionModel(X_train, X_test,  y_train, y_test)

clf2, loss_arr_2, acc_arr_2 = logesticRegressionModel.train(5)

In [ ]:
test_acc2, test_loss2, report2, matrix2  = logesticRegressionModel.test()
